Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе (подбирайте параметры, ориентируясь на качество, а не на результаты, которые вы получали без tfidf);

8) в отдельной ячейке сравните лучшую модель без tfidf и лучшую модель с tfidf (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте темы лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **


Острожнее интерпретируйте полученные результаты. Если один алгоритм сработал хорошо в этом задании - не значит, что он всегда будет хорошо работать, и наоборот.

## 1.

In [221]:
from string import punctuation as punct
import re
import gensim
import pymorphy2
import numpy as np
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words

In [305]:
accents = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}

In [306]:
punct += '…«»–'

In [307]:
morph = pymorphy2.MorphAnalyzer()

In [308]:
stop_words = get_stop_words('ru')

In [309]:
with open('wiki_data.txt') as f:
    texts = f.readlines()[:10000]

In [310]:
def normalize_text(text):
    tokens = [token.strip(punct) for token 
              in word_tokenize(text, language='russian') 
              if re.search(r'[a-zа-яё]', token, re.I)]
    tokens_without_accents = [''.join(accents.get(ch, ch) for ch in token.lower()) 
                              for token in tokens] # чтобы убрать ударения 
    lemmas = [morph.parse(token)[0].normal_form for token in tokens_without_accents]
    lemmas_without_stops = [lemma for lemma in lemmas if lemma not in stop_words]
    return lemmas_without_stops

In [311]:
texts = [normalize_text(text) for text in texts]

## 2.

In [312]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) 
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

## 3.

In [313]:
ngrammed_texts = list(ngrammed_texts)

In [314]:
for i, text in enumerate(ngrammed_texts):
    ngrammed_texts[i] = [word for word in text if len(word) > 2]

In [315]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [316]:
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

## 4.

In [371]:
ngrammed_texts = [ngrammed_text for ngrammed_text in ngrammed_texts if ngrammed_text]

In [372]:
corpus = [dictionary.doc2bow(ngrammed_text) for ngrammed_text in ngrammed_texts]

In [118]:
ldas = {}

In [127]:
ldas['lda_1'] = gensim.models.LdaMulticore(corpus,
                                           num_topics=20,
                                           id2word=dictionary,
                                           passes=30)

In [128]:
ldas['lda_2'] = gensim.models.LdaMulticore(corpus,
                                           num_topics=20,
                                           alpha='asymmetric',
                                           id2word=dictionary,
                                           passes=30)

In [129]:
ldas['lda_3'] = gensim.models.LdaMulticore(corpus,
                                           num_topics=20,
                                           alpha=10,
                                           id2word=dictionary,
                                           passes=30)

In [130]:
ldas['lda_4'] = gensim.models.LdaMulticore(corpus,
                                           num_topics=20,
                                           alpha=0.001,
                                           id2word=dictionary,
                                           passes=30)

In [131]:
ldas['lda_5'] = gensim.models.LdaMulticore(corpus,
                                           num_topics=20,
                                           eta=10,
                                           id2word=dictionary,
                                           passes=30)

In [132]:
ldas['lda_6'] = gensim.models.LdaMulticore(corpus,
                                           num_topics=20,
                                           eta=0.001,
                                           id2word=dictionary,
                                           passes=30)

In [133]:
def pretty_print_topics(lda):
    for topic in sorted(lda.print_topics(num_topics=100, num_words=7)):
        topic_words = topic[1].replace('_', ' ')
        print(topic[0] + 1, end='. ')
        print(*re.findall(r'"(.+?)"', topic_words), sep=', ', end='\n\n')

In [134]:
for lda in ldas:
    print(lda)
    pretty_print_topics(ldas[lda])

lda_1
1. войско, армия, фронт, посёлок, станция, дивизия, улица

2. хутор, поселение, входить состав, район ростовский, право, ростовский область, сельский поселение

3. село, остров, река, корабль, экспедиция, северный, озеро

4. фильм, роль, сша, сериал, американский, актёр, сезон

5. олимпийский игра, свой история, игра, завоевать медаль, летний олимпийский, принимать участие, участие летний

6. игра, компания, версия, использовать, серия, выпустить, система

7. площадь км², составлять почтовый, индекс телефонный, код занимать, основать находиться, код коатуа, село

8. автомобиль, подвеска, модель, значение, флаг, колесо, использоваться

9. система, должный, случай, результат, проект, исследование, использовать

10. фильм, книга, доктор, роман, слово, говорить, отец

11. группа, песня, альбом, музыка, музыкальный, театр, выпустить

12. вид, растение, некоторый, встречаться, семейство, лист, форма

13. церковь, здание, дом, храм, музей, улица, художник

14. страна, король, территория

## 5.

На мой взгляд, лучше всего получилась `lda_4` с параметром `alpha=0.001`:

In [135]:
pretty_print_topics(ldas['lda_4'])

1. игра, группа, альбом, песня, выпустить, версия, the

2. турнир, король, открытый чемпионат, финал, выиграть, пара, пешка

3. страна, право, государство, россия, закон, общество, сша

4. армия, войско, фронт, война, дивизия, август, военный

5. дело, суд, москва, убийство, убить, арестовать, март

6. корабль, самолёт, война, флот, войско, отряд, военный

7. система, автомобиль, использовать, использоваться, модель, тип, значение

8. остров, река, озеро, территория, северный, вода, экспедиция

9. индекс телефонный, составлять почтовый, площадь км², код занимать, район житомирский, основать находиться, занимать площадь

10. университет, институт, школа, профессор, окончить, ссср, книга

11. фильм, книга, роль, серия, доктор, ребёнок, отец

12. церковь, сын, храм, король, святой, здание, монастырь

13. клуб, команда, сезон, матч, сборная, выступать, игра

14. партия, член, совет, президент, правительство, выборы, политический

15. вид, растение, семейство, лист, встречаться, животное, н

Лучшие три темы из нее:

1. игра, группа, альбом, песня, выпустить, версия, the
13. клуб, команда, сезон, матч, сборная, выступать, игра
20. олимпийский игра, свой история, игра, завоевать медаль, летний олимпийский, принимать участие, участие летний

## 6. 

In [302]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

## 7. 

In [211]:
ldas_tfidf = {}

In [212]:
ldas_tfidf['lda_1'] = gensim.models.LdaMulticore(corpus,
                                                 num_topics=20,
                                                 id2word=dictionary,
                                                 passes=30)

In [213]:
ldas_tfidf['lda_2'] = gensim.models.LdaMulticore(corpus,
                                                 num_topics=20,
                                                 alpha='asymmetric',
                                                 id2word=dictionary,
                                                 passes=30)

In [214]:
ldas_tfidf['lda_3'] = gensim.models.LdaMulticore(corpus,
                                                 num_topics=20,
                                                 alpha=10,
                                                 id2word=dictionary,
                                                 passes=30)

In [215]:
ldas_tfidf['lda_4'] = gensim.models.LdaMulticore(corpus,
                                                 num_topics=20,
                                                 alpha=0.001,
                                                 id2word=dictionary,
                                                 passes=30)

In [216]:
ldas_tfidf['lda_5'] = gensim.models.LdaMulticore(corpus,
                                                 num_topics=20,
                                                 eta=10,
                                                 id2word=dictionary,
                                                 passes=30)

In [217]:
ldas_tfidf['lda_6'] = gensim.models.LdaMulticore(corpus,
                                                 num_topics=20,
                                                 eta=0.001,
                                                 id2word=dictionary,
                                                 passes=30)

In [218]:
for lda in ldas_tfidf:
    print(lda)
    pretty_print_topics(ldas_tfidf[lda])

lda_1
1. олимпийский игра, свой история, завоевать медаль, летний олимпийский, участие летний, игра, зимний олимпийский

2. фамилия известный, носитель, мюнхен фрг, меридиан восточный, тепловоз, пуэрто-рико принимать, ямайка принимать

3. улица, различный населить, родиться чешуекрылый, азербайджан, ссср, армения, остров

4. южный корея, вещество, турнир, открытый чемпионат, родиться подсемейство, белок, словакия

5. армия, война, дивизия, фронт, войско, командир, бой

6. жук, округ, радужница, таджикистан, вид жук-листоед, хутор аксайский, надкрылья

7. почтовый индекс, телефонный код, занимать площадь, коатуа население, область код, перепись составлять, км² житомирский

8. романовский район, романовский р-н, литература, емильчинский район, емильчинский р-н, ивановка, село

9. составлять почтовый, индекс телефонный, код занимать, площадь км², основать находиться, код коатуа, коростенский район

10. альбом, песня, группа, выпустить, the, игра, уезд

11. село, сельский совет, харьковски

## 8. 

In [197]:
pretty_print_topics(ldas['lda_4'])

1. игра, группа, альбом, песня, выпустить, версия, the

2. турнир, король, открытый чемпионат, финал, выиграть, пара, пешка

3. страна, право, государство, россия, закон, общество, сша

4. армия, войско, фронт, война, дивизия, август, военный

5. дело, суд, москва, убийство, убить, арестовать, март

6. корабль, самолёт, война, флот, войско, отряд, военный

7. система, автомобиль, использовать, использоваться, модель, тип, значение

8. остров, река, озеро, территория, северный, вода, экспедиция

9. индекс телефонный, составлять почтовый, площадь км², код занимать, район житомирский, основать находиться, занимать площадь

10. университет, институт, школа, профессор, окончить, ссср, книга

11. фильм, книга, роль, серия, доктор, ребёнок, отец

12. церковь, сын, храм, король, святой, здание, монастырь

13. клуб, команда, сезон, матч, сборная, выступать, игра

14. партия, член, совет, президент, правительство, выборы, политический

15. вид, растение, семейство, лист, встречаться, животное, н

In [219]:
pretty_print_topics(ldas_tfidf['lda_1'])

1. олимпийский игра, свой история, завоевать медаль, летний олимпийский, участие летний, игра, зимний олимпийский

2. фамилия известный, носитель, мюнхен фрг, меридиан восточный, тепловоз, пуэрто-рико принимать, ямайка принимать

3. улица, различный населить, родиться чешуекрылый, азербайджан, ссср, армения, остров

4. южный корея, вещество, турнир, открытый чемпионат, родиться подсемейство, белок, словакия

5. армия, война, дивизия, фронт, войско, командир, бой

6. жук, округ, радужница, таджикистан, вид жук-листоед, хутор аксайский, надкрылья

7. почтовый индекс, телефонный код, занимать площадь, коатуа население, область код, перепись составлять, км² житомирский

8. романовский район, романовский р-н, литература, емильчинский район, емильчинский р-н, ивановка, село

9. составлять почтовый, индекс телефонный, код занимать, площадь км², основать находиться, код коатуа, коростенский район

10. альбом, песня, группа, выпустить, the, игра, уезд

11. село, сельский совет, харьковский обла

Перплексия показывает насколько хороше моделируется корпус. Чем ближе к нулю, тем лучше. Можно использовать, чтобы настраивать количество проходов по корпусу (когда перестало улучшаться, то можно останавливаться).

In [318]:
best_lda_perplexity = ldas['lda_4'].log_perplexity(corpus)
best_lda_tfidf_perplexity = ldas_tfidf['lda_1'].log_perplexity(corpus)

In [319]:
print(f'Перплексия лучшей модели без TF-IDF: {best_lda_perplexity}')
print(f'Перплексия лучшей модели с TF-IDF: {best_lda_tfidf_perplexity}')

Перплексия лучшей модели без TF-IDF: -8.097141685172714
Перплексия лучшей модели с TF-IDF: -9.107181191402598


Ещё есть когерентность. Она численно оценивает качество тем (проверяется, что темы состоят из разных слов и что в теме есть топ тематических слов). 

In [373]:
np.seterr(divide='warn', invalid='warn')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [379]:
[text for text in ngrammed_texts if not text]

[]

In [374]:
def lda_coherence(lda):
    topics = []
    for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics,
                                                       texts=texts,
                                                       dictionary=dictinary,
                                                       coherence='c_v')
    return coherence_model_lda.get_coherence()

In [375]:
best_lda_coherence = lda_coherence(ldas['lda_4'])

/Users/liliooon/miniforge3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/Users/liliooon/miniforge3/lib/python3.9/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [376]:
best_lda_tfidf_coherence = lda_coherence(ldas_tfidf['lda_1'])

In [377]:
print(f'Когерентность лучшей модели без TF-IDF: {best_lda_coherence}')
print(f'Когерентность лучшей модели с TF-IDF: {best_lda_tfidf_coherence}')

Когерентность лучшей модели без TF-IDF: nan
Когерентность лучшей модели с TF-IDF: nan


In [366]:
best_lda_coherence = gensim.models.CoherenceModel(model=ldas['lda_4'],
                                                  corpus=corpus,
                                                  texts=texts,
                                                  coherence='c_v').get_coherence()

/Users/liliooon/miniforge3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/Users/liliooon/miniforge3/lib/python3.9/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [369]:
best_lda_tfidf_coherence = gensim.models.CoherenceModel(model=ldas_tfidf['lda_1'],
                                                        texts=texts,
                                                        dictionary=dictinary,
                                                        coherence='c_v').get_coherence()

/Users/liliooon/miniforge3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/Users/liliooon/miniforge3/lib/python3.9/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [370]:
print(f'Когерентность лучшей модели без TF-IDF: {best_lda_coherence}')
print(f'Когерентность лучшей модели с TF-IDF: {best_lda_tfidf_coherence}')

Когерентность лучшей модели без TF-IDF: nan
Когерентность лучшей модели с TF-IDF: nan


Чем выше, тем лучше.

## 9. 

In [23]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [24]:
stexts = [' '.join(text) for text in texts]

In [25]:
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, max_df=0.1, ngram_range=(1,2))
X = vectorizer.fit_transform(stexts)

In [26]:
# n_components - главный параметр в NMF, это количество тем. 
# Если данных много, то увеличения этого параметра сильно увеличивает время обучения
model = NMF(n_components=100)

In [27]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=100, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [28]:
model.components_.shape # матрица темы на слова

(100, 2000)

In [29]:
model.transform(X).shape # матрица документы на темы

(10000, 100)

In [30]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [31]:
feat_names = vectorizer.get_feature_names()

In [32]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "--".join(words))

0 летний олимпийский--летний--олимпийский игра--участие летний
1 украина основать--год находиться--области код--составлять человек
2 век--xix--xix век--xx
3 расстояние километр--харьковский--расстояние--километр расположить
4 зимний олимпийский--зимний--олимпийский игра--участие зимний
5 украина находиться--области код--составлять человек--ул
6 новоград--новоград волынский--волынский--области население
7 клуб--матч--лига--забить
8 хутор--ростовский--район ростовский--ростовский области
9 ул--наш ул--карл--области население
10 род--подсемейство--около--некоторый
11 сын--отец--жена--ребёнок
12 овручский--области код--сергей--13
13 остров--относиться--километр--пролив
14 емильчинский--области код--украина находиться--23
15 населить пункт--населить--пункт--государство
16 уезд--год уезд--специальный район--округ
17 хорошевский--области население--украина находиться--украина основать
18 альбом--the--of--выпустить
19 коростенский--области код--украина основать--год находиться
20 фильм--режисс

## 10.

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью BigARTM**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор) 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases, можно также попробовать другие способы построить нграммы); 

3) сохраните тексты .vw формате;

4) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова, сохраните словарь и посмотрите на него, вдруг что-то плохое сразу будет заметно - из словаря можно просто вручную или правилом удалять строки, при загрузке ничего не сломается); 

5) постройте несколько ARTM моделей (переберите количество тем, поменяйте значения tau у регуляризаторов), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

6) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы

7) в другой ячейке нарисуйте график обучения этой модели 

8) в третьей ячейки опишите какие параметры (количество тем, регуляризаторы, их tau) вы использовали и как обучали (например, после скольки проходов добавили регуляризатор разрежнивания тем (Phi), добавляли ли разреженность документам (Theta) и когда, как повышали значения, сколько итерации модель продожала улучшаться (снижалась перплексия, росли другие метрики);

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 6, 7 и 8. (3, 1, 4 баллов соответственно). Чтобы заработать остальные 2 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **